<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Загрузим-и-изучим-данные" data-toc-modified-id="Загрузим-и-изучим-данные-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Загрузим и изучим данные</a></span></li><li><span><a href="#Явные-дубликаты" data-toc-modified-id="Явные-дубликаты-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Явные дубликаты</a></span></li><li><span><a href="#Лемматизация" data-toc-modified-id="Лемматизация-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Лемматизация</a></span></li><li><span><a href="#Разобьем-данные-на-выборки" data-toc-modified-id="Разобьем-данные-на-выборки-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Разобьем данные на выборки</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#TfidfVectorizer" data-toc-modified-id="TfidfVectorizer-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>TfidfVectorizer</a></span><ul class="toc-item"><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li><li><span><a href="#Дерево-решений" data-toc-modified-id="Дерево-решений-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>Дерево решений</a></span></li></ul></li><li><span><a href="#CountVectorizer" data-toc-modified-id="CountVectorizer-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>CountVectorizer</a></span><ul class="toc-item"><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li><li><span><a href="#Дерево-решений" data-toc-modified-id="Дерево-решений-2.2.2"><span class="toc-item-num">2.2.2&nbsp;&nbsp;</span>Дерево решений</a></span></li></ul></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import nltk
import numpy as np
import pandas as pd
import re 

from nltk.corpus import stopwords as nltk_stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer


from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier

nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### Загрузим и изучим данные

Загрузим данные

In [2]:
comments = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv')

In [3]:
comments.info()
comments.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


### Явные дубликаты

Посмотрим имеются ли явные дубликаты 

In [4]:
comments.duplicated().sum()

0

### Лемматизация

Выделим леммы в столбце "text", оставив в тексте только английские символы и пробелы

In [5]:
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

In [6]:
def lemmatize(text):
    lemmatizer = WordNetLemmatizer()
    text = re.sub(r"[^a-zA-Z']", ' ', text).split()
    text = " ".join(text)
    text = text.lower()
    lemm_list = nltk.word_tokenize(text)
    lemm_text = ' '.join([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in lemm_list])

    return lemm_text

In [7]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [8]:
%%time

comments['lemma'] = comments['text'].apply(lemmatize)

CPU times: user 16min 49s, sys: 1min 38s, total: 18min 28s
Wall time: 18min 29s


In [9]:
comments.head()

,text,toxic,lemma
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edits make under my userna...
1,D'aww! He matches this background colour I'm s...,0,d'aww he match this background colour i 'm see...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man i 'm really not try to edit war it 's ...
3,"""\nMore\nI can't make any real suggestions on ...",0,more i ca n't make any real suggestion on impr...
4,"You, sir, are my hero. Any chance you remember...",0,you sir be my hero any chance you remember wha...


### Разобьем данные на выборки

Разделим исходные данные на обучающую и тестовую выборки.

In [10]:
features_train, features_test, target_train, target_test = train_test_split(comments['lemma'], comments['toxic'], test_size=0.25, random_state=12345)

print('Размер обучающей выборки:', features_train.shape, target_train.shape)
print('Размер тестовой выборки:', features_test.shape, target_test.shape)

Размер обучающей выборки: (119678,) (119678,)
Размер тестовой выборки: (39893,) (39893,)


<font color='blue'><b>Комментарий ревьюера: </b></font> ✔️\
<font color='green'> Здорово , что у нас есть выборка для тестов!</font>

### Вывод

Загрузили данные. Провели лемматизацию данных и разбили на выборки, для дальнейшего анализа

## Обучение

In [11]:
comments_f1 = pd.DataFrame({'Class': [], 'Model': [], 'F1': []}) 

Загрузим список стоп-слов:

In [12]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### TfidfVectorizer

In [13]:
tf_idf = TfidfVectorizer(stop_words=stopwords)

tf_idf_train = tf_idf.fit_transform(features_train)
tf_idf_test = tf_idf.transform(features_test)

print('Размер матрицы:', tf_idf_train.shape)

Размер матрицы: (119678, 128561)


#### Логистическая регрессия

In [14]:
%%time

regressor = LogisticRegression()
hyperparams = [{'max_iter':(100, 1000), 'class_weight':['balanced', None], 'solver':['liblinear'], 'random_state':[12345]}]

cv = GridSearchCV(regressor, hyperparams, cv=5, scoring='f1')
cv.fit(tf_idf_train, target_train)

print('Лучшая модель:', cv.best_params_)

Лучшая модель: {'class_weight': 'balanced', 'max_iter': 100, 'random_state': 12345, 'solver': 'liblinear'}
CPU times: user 1min 55s, sys: 1min 55s, total: 3min 50s
Wall time: 3min 52s


In [15]:
model = LogisticRegression(class_weight='balanced', max_iter=100, random_state=12345, solver='liblinear')
model.fit(tf_idf_train, target_train)
predicted = model.predict(tf_idf_test)
f1 = f1_score(target_test, predicted)

comments_f1 = comments_f1.append({'Class':'TfidfVectorizer', 'Model':'LogisticRegression', 'F1':f1}, ignore_index=True)
comments_f1.iloc[-1]

Class       TfidfVectorizer
Model    LogisticRegression
F1                  0.74976
Name: 0, dtype: object

#### Дерево решений

In [16]:
%%time

regressor = DecisionTreeClassifier() 
max_depth_list = [x for x in range(1, 11)]
hyperparams = [{'max_depth':max_depth_list, 'random_state':[12345]}]

cv = GridSearchCV(regressor, hyperparams, scoring='f1', cv=5)
cv.fit(tf_idf_train, target_train)
print('Лучшая модель:', cv.best_params_)

Лучшая модель: {'max_depth': 10, 'random_state': 12345}
CPU times: user 7min 16s, sys: 1.2 s, total: 7min 17s
Wall time: 7min 20s


In [17]:
model = DecisionTreeClassifier(max_depth=10, random_state=12345)
model.fit(tf_idf_train, target_train)
predicted = model.predict(tf_idf_test)
f1 = f1_score(target_test, predicted)

comments_f1 = comments_f1.append({'Class':'TfidfVectorizer', 'Model':'DecisionTreeClassifier', 'F1':f1}, ignore_index=True)
comments_f1.iloc[-1]

Class           TfidfVectorizer
Model    DecisionTreeClassifier
F1                     0.611844
Name: 1, dtype: object

### CountVectorizer

In [18]:
vect = CountVectorizer(stop_words=stopwords)

vect_train = vect.fit_transform(features_train)
vect_test = vect.transform(features_test)

print('Размер матрицы:', vect_train.shape)

Размер матрицы: (119678, 128561)


#### Логистическая регрессия

In [19]:
%%time

regressor = LogisticRegression()
hyperparams = [{'max_iter':[1000], 'class_weight':['balanced', None], 'solver':['lbfgs', 'liblinear'], 'random_state':[12345]}]

cv = GridSearchCV(regressor, hyperparams, cv=2, scoring='f1', n_jobs=1)
cv.fit(vect_train, target_train)

print('Лучшая модель:', cv.best_params_)

Лучшая модель: {'class_weight': 'balanced', 'max_iter': 1000, 'random_state': 12345, 'solver': 'lbfgs'}
CPU times: user 19min 41s, sys: 22min 1s, total: 41min 42s
Wall time: 41min 48s


In [20]:
model = LogisticRegression(class_weight='balanced', max_iter=1000, random_state=12345, solver='lbfgs')
model.fit(vect_train, target_train)
predicted = model.predict(vect_test)
f1 = f1_score(target_test, predicted)

comments_f1 = comments_f1.append({'Class':'CountVectorizer', 'Model':'LogisticRegression', 'F1':f1}, ignore_index=True)
comments_f1.iloc[-1]

Class       CountVectorizer
Model    LogisticRegression
F1                  0.75801
Name: 2, dtype: object

#### Дерево решений

In [22]:
%%time

regressor = DecisionTreeClassifier() 
max_depth_list = [x for x in range(1, 11)]
hyperparams = [{'max_depth':max_depth_list, 'random_state':[12345]}]

cv = GridSearchCV(regressor, hyperparams, scoring='f1', cv=5)
cv.fit(vect_train, target_train)
print('Лучшая модель:', cv.best_params_)

Лучшая модель: {'max_depth': 10, 'random_state': 12345}
CPU times: user 6min 3s, sys: 248 ms, total: 6min 3s
Wall time: 6min 5s


In [23]:
model = DecisionTreeClassifier(max_depth=10, random_state=12345)
model.fit(vect_train, target_train)
predicted = model.predict(vect_test)
f1 = f1_score(target_test, predicted)

comments_f1 = comments_f1.append({'Class':'CountVectorizer', 'Model':'DecisionTreeClassifier', 'F1':f1}, ignore_index=True)
comments_f1.iloc[-1]

Class           CountVectorizer
Model    DecisionTreeClassifier
F1                     0.597555
Name: 3, dtype: object

## Выводы

In [24]:
def highlight_max(s, props=''):
    return np.where(s == np.nanmax(s.values), props, '')

comments_f1.style.apply(highlight_max, props='color:white; font-weight:bold; background-color:darkblue;', axis=0)

,Class,Model,F1
0,TfidfVectorizer,LogisticRegression,0.749760
1,TfidfVectorizer,DecisionTreeClassifier,0.611844
2,CountVectorizer,LogisticRegression,0.758010
3,CountVectorizer,DecisionTreeClassifier,0.597555


Была найдена необходимая модель, с наилучшей метрикой. Это модель LogisticRegression с классом CountVectorizer и метрикой 0.758010